## Remember we are running this in Notebook with PySpark created in AWS EMR cluster.

In [1]:
# Importing Pyspark and other necessary modules
import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession



VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
3,application_1589215074060_0004,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
from pyspark.sql.types import *

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Starting Spark application

In [3]:
spark = SparkSession \
    .builder \
    .appName("Netflix Movie Recommendation System") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
file_path = 's3://aws-emr-resources-333610833165-us-east-1/TrainingRatings.txt'

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
schema1 = StructType([
    StructField("MovieID", IntegerType(), True),
    StructField("UserID", IntegerType(), True),
    StructField("Ratings", DoubleType(), True)])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
df_train = spark.read \
 .schema(schema1) \
 .option("inferSchema", "True") \
 .csv(file_path)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
df_train.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- MovieID: integer (nullable = true)
 |-- UserID: integer (nullable = true)
 |-- Ratings: double (nullable = true)

In [8]:
df_train.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-------+-------+
|MovieID| UserID|Ratings|
+-------+-------+-------+
|      8|1744889|    1.0|
|      8|1395430|    2.0|
|      8|1205593|    4.0|
|      8|1488844|    4.0|
|      8|1447354|    1.0|
+-------+-------+-------+
only showing top 5 rows

In [9]:
df_train.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

3255352

In [10]:
df_train.describe().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------------------+-----------------+------------------+
|summary|           MovieID|           UserID|           Ratings|
+-------+------------------+-----------------+------------------+
|  count|           3255352|          3255352|           3255352|
|   mean|  8724.66010434509|1327058.466035931| 3.481187595074204|
| stddev|5107.4015117381605|762688.6901278322|1.0828732789156674|
|    min|                 8|                7|               1.0|
|    max|             17742|          2649285|               5.0|
+-------+------------------+-----------------+------------------+

## Using Model-based approach for Collaborative filtering

### Alternative Least Squares model

In [11]:
from pyspark.ml.recommendation import ALS

# Build the recommendation model using ALS on the training data
als = ALS(maxIter=5, regParam=0.01, userCol="UserID", itemCol="MovieID", ratingCol="Ratings")
model = als.fit(df_train)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
# Getting test data into another dataframe
schema2 = StructType([
    StructField("MovieID", IntegerType(), True),
    StructField("UserID", IntegerType(), True),
    StructField("Ratings", DoubleType(), True)])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
file_path_test = 's3://aws-emr-resources-333610833165-us-east-1/TestingRatings.txt'

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
#Building Testing Dataframe
df_test = spark.read \
 .schema(schema2) \
 .option("inferSchema", "True") \
 .csv(file_path_test)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
df_test.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-------+-------+
|MovieID| UserID|Ratings|
+-------+-------+-------+
|      8| 573364|    1.0|
|      8|2149668|    3.0|
|      8|1089184|    3.0|
|      8|2465894|    3.0|
|      8| 534508|    1.0|
+-------+-------+-------+
only showing top 5 rows

In [16]:
df_test.count()
#100,478 rows in total

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

100478

### Finding Distict Movies and Users in the testing dataset

In [17]:
# getting the count of unique Users
df_uniqueUsers = df_test.groupBy(['UserID']).count()
df_uniqueUsers.describe().show()
# Unique count = 27555

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------------------+------------------+
|summary|            UserID|             count|
+-------+------------------+------------------+
|  count|             27555|             27555|
|   mean|1325637.6160406459|3.6464525494465616|
| stddev| 762738.5020270854| 2.351318323080286|
|    min|                 7|                 1|
|    max|           2649285|                70|
+-------+------------------+------------------+

In [18]:
# getting the count of unique Movies in testing data
df_uniqueMovies = df_test.groupBy(['MovieID']).count()
df_uniqueMovies.describe().show()
#Unique count = 1701

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-----------------+------------------+
|summary|          MovieID|             count|
+-------+-----------------+------------------+
|  count|             1701|              1701|
|   mean| 8919.66784244562| 59.06995884773662|
| stddev|5168.709499952632|121.58856049833442|
|    min|                8|                 1|
|    max|            17742|               811|
+-------+-----------------+------------------+

### Making predictions from Test data

In [19]:
predictions = model.transform(df_test)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
predictions.orderBy('prediction',ascending=False).show()
#Top 20 predictions made by the model.

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-------+-------+----------+
|MovieID| UserID|Ratings|prediction|
+-------+-------+-------+----------+
|   2939|1374515|    1.0|  6.639826|
|  16022| 657521|    5.0| 5.8767805|
|   1848| 761430|    5.0| 5.8063545|
|   2532|2302897|    5.0| 5.7852683|
|  14013| 761430|    5.0| 5.7831683|
|  12232|2617068|    5.0|   5.70807|
|  12298| 685113|    5.0|  5.668999|
|    577|1958918|    2.0| 5.6602597|
|  12232|2082805|    5.0| 5.6438236|
|  15836|1789629|    5.0|  5.639774|
|  12293| 304581|    5.0|  5.625397|
|  16902|   5980|    3.0| 5.5962944|
|   8678|2539096|    5.0| 5.5819926|
|   7382|1462959|    5.0|  5.574134|
|  11563|2134334|    2.0| 5.5598297|
|  10080| 295220|    5.0| 5.5554013|
|   3290| 123974|    5.0| 5.5488434|
|  12298| 179995|    5.0|  5.548734|
|   3824|  38244|    5.0| 5.5438766|
|  15507| 203576|    5.0|  5.539093|
+-------+-------+-------+----------+
only showing top 20 rows

### Evaluating the predictions made using metrics : Root Mean Squared Error and Absolute Mean Error

In [21]:
from pyspark.ml.evaluation import RegressionEvaluator

evaluator = RegressionEvaluator(metricName="rmse", labelCol="Ratings",predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root means quare error = " + str(rmse))


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Root means quare error = 0.8532122359688417

In [22]:
evaluator_2 = RegressionEvaluator(metricName="mae", labelCol="Ratings",predictionCol="prediction")
mae = evaluator_2.evaluate(predictions)
print("Mean Absolute Error = " + str(mae))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Mean Absolute Error = 0.6702405307684796

In [23]:
"""Adding a new user, say me. Let the UserID be '1' and let's say I have watched 10 movies and did rating them.I have manually 
added in Training set and creating a new dataframe. I'll be outputing predictions for the movies I haven't watched. """

#I have manually added the 10 rows in the Training set with User ID '1', 10 Ratings to 10 different MovieID's.
#I have took the help of Movies_titles.txt for knowing the each Movie name for each ID and rated them.

file_path_addedUserMe = 's3://aws-emr-resources-333610833165-us-east-1/TrainingRatings_addedUser1.txt'

#I have created a new dataframe including the user 'Me' and performing same operation
df_train_addedUserMe = spark.read \
 .schema(schema1) \
 .option("inferSchema", "True") \
 .csv(file_path_addedUserMe)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
df_train_addedUserMe.describe().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-----------------+------------------+------------------+
|summary|          MovieID|            UserID|           Ratings|
+-------+-----------------+------------------+------------------+
|  count|          3255362|           3255362|           3255362|
|   mean| 8724.63380048056|1327054.3895078336|3.4811864241211885|
| stddev|5107.415718447187| 762691.0652031554|1.0828736515089445|
|    min|                8|                 1|               1.0|
|    max|            17742|           2649285|               5.0|
+-------+-----------------+------------------+------------------+

In [30]:
user_Me = df_train_addedUserMe.filter(df_train_addedUserMe['UserID']==1).select(['UserID','MovieID','Ratings'])
user_Me.show()
#The values below are manually defined in the Training Set.

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+-------+-------+
|UserID|MovieID|Ratings|
+------+-------+-------+
|     1|     69|    4.0|
|     1|     91|    4.0|
|     1|     76|    3.0|
|     1|    129|    2.0|
|     1|    152|    1.0|
|     1|    183|    3.0|
|     1|    192|    4.0|
|     1|    204|    5.0|
|     1|    233|    3.0|
|     1|    289|    2.0|
+------+-------+-------+

In [26]:
#Training new model with Training set added with User_Me
model_2 = als.fit(df_train_addedUserMe)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
recommendations = model_2.transform(df_test)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
#Obtaining the top 10 recommendations for the user Me 

recommendations.orderBy('prediction',ascending=False).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-------+-------+----------+
|MovieID| UserID|Ratings|prediction|
+-------+-------+-------+----------+
|    218|2302897|    4.0|  6.427806|
|   9612| 211185|    4.0|  6.232914|
|  14320| 424513|    5.0| 6.1172304|
|  14506| 827378|    5.0| 5.9151344|
|  11563| 794999|    5.0|  5.866231|
|   2682|2180836|    1.0|  5.848472|
|  14283|2573207|    5.0| 5.7979774|
|   2290|1303007|    5.0| 5.7765975|
|   2861| 409104|    5.0|  5.769668|
|   1848| 761430|    5.0|  5.745669|
|  16022| 657521|    5.0|  5.739973|
|   3824|1144718|    4.0| 5.7019215|
|  15836|1789629|    5.0|  5.685099|
|   7148|1944765|    1.0| 5.6831737|
|  12293| 304581|    5.0|  5.677288|
|   3147|  49890|    5.0|  5.676237|
|  15051|1938727|    4.0|  5.666596|
|   2861|2156560|    5.0|  5.655134|
|    577| 838612|    5.0|  5.618471|
|   8561| 685113|    5.0|  5.604727|
+-------+-------+-------+----------+
only showing top 20 rows

 The first Movie recommended by this system has a MovieID '218' so I can map to Movie_titles.txt to get the movie details.

In [31]:
# Getting Movie titles into another dataframe
schema3 = StructType([
    StructField("MovieID", IntegerType(), True),
    StructField("YearOfRelease", IntegerType(), True),
    StructField("MovieName", StringType(), True)])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [32]:
movies_titles_df = spark.read \
 .schema(schema3) \
 .option("inferSchema", "True") \
 .csv('s3://aws-emr-resources-333610833165-us-east-1/movie_titles.txt')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [33]:
movies_titles_df.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-------------+--------------------+
|MovieID|YearOfRelease|           MovieName|
+-------+-------------+--------------------+
|      1|         2003|     Dinosaur Planet|
|      2|         2004|Isle of Man TT 20...|
|      3|         1997|           Character|
|      4|         1994|Paula Abdul's Get...|
|      5|         2004|The Rise and Fall...|
+-------+-------------+--------------------+
only showing top 5 rows

In [35]:
 movies_titles_df.filter(movies_titles_df['MovieID']==218).select(['YearOfRelease','MovieName']).show()
# Triumph is the first movie recommended to me with high prediction Rating.  

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+--------------------+
|YearOfRelease|           MovieName|
+-------------+--------------------+
|         1983|Triumph: Live at ...|
+-------------+--------------------+